![AIcrowd-Logo](https://raw.githubusercontent.com/AIcrowd/AIcrowd/master/app/assets/images/misc/aicrowd-horizontal.png)


# Getting Started Code for [Chess Configuration](https://www.aicrowd.com/challenges/ai-blitz-6/problems/chess-configuration) on AIcrowd
#### Author : Shubhamai

## Download Necessary Packages 📚

In this baseline we are going to use FastAI as our main library to train out model and making & submitting predictions

In [1]:
!pip install --upgrade fastai git+https://gitlab.aicrowd.com/yoogottamk/aicrowd-cli.git >/dev/null
%load_ext aicrowd.magic

  Running command git clone -q https://gitlab.aicrowd.com/yoogottamk/aicrowd-cli.git /tmp/pip-req-build-dc1w59pj


## Download Data
The first step is to download out train test data. We will be training a model on the train data and make predictions on test data. We submit our predictions.


In [2]:
API_KEY = 'b46152c118edae059e08ebd1f8ada78f' #Please enter your API Key [https://www.aicrowd.com/participants/me]
%aicrowd login --api-key $API_KEY

Verifying API Key...
API Key valid
Saved API Key successfully!


In [3]:
%aicrowd dataset download --challenge chess-configuration -j 3

In [4]:
!rm -rf data
!mkdir data

!unzip train.zip  -d data/ 
!unzip val.zip -d data/ 
!unzip test.zip  -d data/ 

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

Streaming output truncated to the last 5000 lines.
  inflating: data/test/5499.jpg      
  inflating: data/test/55.jpg        
  inflating: data/test/550.jpg       
  inflating: data/test/5500.jpg      
  inflating: data/test/5501.jpg      
  inflating: data/test/5502.jpg      
  inflating: data/test/5503.jpg      
  inflating: data/test/5504.jpg      
  inflating: data/test/5505.jpg      
  inflating: data/test/5506.jpg      
  inflating: data/test/5507.jpg      
  inflating: data/test/5508.jpg      
  inflating: data/test/5509.jpg      
  inflating: data/test/551.jpg       
  inflating: data/test/5510.jpg      
  inflating: data/test/5511.jpg      
  inflating: data/test/5512.jpg      
  inflating: data/test/5513.jpg      
  inflating: data/test/5514.jpg      
  inflating: data/test/5515.jpg      
  inflating: data/test/5516.jpg      
  inflating: data/test/5517.jpg      
  inflating: data/test/5518.jpg      
  inflating: data/test/5519.jpg      
  inflating: data/test/552.jpg       


## Import packages

In [5]:
import pandas as pd
from fastai.vision.all import *
import os
from tqdm.notebook import tqdm

## Load Data
- We use pandas 🐼 library to load our data.   
- Pandas loads the data into dataframes and facilitates us to analyse the data.   
- Learn more about it [here](https://www.tutorialspoint.com/python_data_science/python_pandas.htm) 🤓

In [6]:
sample_submission = pd.read_csv("data/sample_submission.csv")

## Visualize the data 👀

In [7]:
sample_submission

,ImageID,label
0,0,1nr2b2/p7/2p1r3/R3pp1k/1P2bP1P/1q2N1P1/5KR1/2B5
1,1,2kbR3/8/1RQ3p1/p6p/2P5/N1P3P1/8/3K4
2,2,r7/p1k4q/Pp3n2/2p5/1QP1bP2/3n2b1/2R1B1PP/1N4KR
3,3,r3kb2/4p3/p1nP2pr/2pb1p2/p1P3BP/P1N1K3/1P6/RN2n1R1
4,4,rn2qbnr/1p2pk2/2p1b2p/3p2p1/NPP3PP/p2Pp2B/P2BQP2/R3K1NR
...,...,...
9995,9995,rnbk1bnr/3p3p/1pp3p1/pP4p1/P1N1pq2/R1NP4/2P1PPPP/2Q1KB1R
9996,9996,1nr5/1r2k1p1/Q3PN1p/p1p4P/1b2P3/1P6/2R2PB1/3K2NR
9997,9997,2q2b1r/1rpkp3/p1n1bn1p/5pp1/2P2N2/NQ1P1PP1/PB1KP2P/R4B1R
9998,9998,rn3br1/8/1pk1b1qp/p1PNB1P1/P1P1pPn1/2N5/3P2P1/R2K1BR1


## Creating Random Submission 👀
In below cell, we are going to create a random labels to submit our predictions.

The below code was originally made by [rosettacode](https://rosettacode.org/) and the original source of code is in this [link](https://rosettacode.org/wiki/Generate_random_chess_position#Python)

In [8]:
piece_list = ["R", "N", "B", "Q", "P"]
 
 
def place_kings(brd):
	while True:
		rank_white, file_white, rank_black, file_black = random.randint(0,7), random.randint(0,7), random.randint(0,7), random.randint(0,7)
		diff_list = [abs(rank_white - rank_black),  abs(file_white - file_black)]
		if sum(diff_list) > 2 or set(diff_list) == set([0, 2]):
			brd[rank_white][file_white], brd[rank_black][file_black] = "K", "k"
			break
 
def populate_board(brd, wp, bp):
	for x in range(2):
		if x == 0:
			piece_amount = wp
			pieces = piece_list
		else:
			piece_amount = bp
			pieces = [s.lower() for s in piece_list]
		while piece_amount != 0:
			piece_rank, piece_file = random.randint(0, 7), random.randint(0, 7)
			piece = random.choice(pieces)
			if brd[piece_rank][piece_file] == " " and pawn_on_promotion_square(piece, piece_rank) == False:
				brd[piece_rank][piece_file] = piece
				piece_amount -= 1
 
def fen_from_board(brd):
	fen = ""
	for x in brd:
		n = 0
		for y in x:
			if y == " ":
				n += 1
			else:
				if n != 0:
					fen += str(n)
				fen += y
				n = 0
		if n != 0:
			fen += str(n)
		fen += "/" if fen.count("/") < 7 else ""
	return fen
 
def pawn_on_promotion_square(pc, pr):
	if pc == "P" and pr == 0:
		return True
	elif pc == "p" and pr == 7:
		return True
	return False
 
 
def start():

  board = [[" " for x in range(8)] for y in range(8)]

  piece_amount_white, piece_amount_black = random.randint(0, 15), random.randint(0, 15)
  place_kings(board)
  populate_board(board, piece_amount_white, piece_amount_black)

  fen = fen_from_board(board)

  board = [[" " for x in range(8)] for y in range(8)]

  return fen

In [9]:
predictions = []

for n in tqdm(range(sample_submission.shape[0])):

  predictions.append(start())

## Save the prediction to csv

In [10]:
sample_submission['label'] = predictions
sample_submission

,ImageID,label
0,0,6pb/1nq3Kq/1r2pp2/3b2q1/7R/5kr1/4pp1b/b7
1,1,8/1N6/8/1b3k2/4n3/8/2K3b1/3NP3
2,2,2B5/1b1Kq1RN/q7/4pQQ1/n1p1r2P/3Rbb2/2bb2B1/k1r4Q
3,3,8/8/1pr5/8/1Q4P1/2K5/k7/3Q4
4,4,8/8/4B3/1p2K3/3R2k1/5p2/3q2b1/4n3
...,...,...
9995,9995,1p6/8/3q4/6k1/8/6b1/8/3K2qn
9996,9996,Q7/1Q1r4/Qp4B1/R1q3Q1/2B3Pk/1B2rR2/1R2P3/2K4b
9997,9997,8/4K3/8/8/2k4B/8/8/8
9998,9998,kqN1p3/2b5/R1Q2KQB/1qPn4/B1Q3n1/5r2/B3N1r1/b6q


### 🚧 Note :    
- Do take a look at the submission format.   
- The submission file should contain a header.   
- Follow all submission guidelines strictly to avoid inconvenience.

In [11]:
sample_submission.to_csv("submission.csv", index=False)

You can submit via AIcrowd CLI directly, (which is still in its beta phase 🙃 ). If you face any problems you can submit by downloading the submission file.

In [12]:
%aicrowd submission create -c chess-configuration -f submission.csv

Output()

                                    ╭─────────────────────────╮                                     
                                    │ Successfully submitted! │                                     
                                    ╰─────────────────────────╯                                     
                                          Important links                                           
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-6/submissions/122667              │
│                  │                                                                               │
│  All submissions │ https://www.aicrowd.com/challenges/ai-blitz-6/submissions?my_submissions=true │
│                  │                                                                               │
│      Leaderboard │ https://www.aicrowd.com/challenges/ai-blitz-6/leaderboards            

## To download the generated csv in colab run the below command

In [ ]:
try:
    from google.colab import files
    files.download('submission.csv') 
except:
    print("Option Only avilable in Google Colab")

### Well Done! 👍 We are all set to make a submission and see your name on leaderborad. Let navigate to [challenge page](https://www.aicrowd.com/challenges/ai-blitz-6/problems/chess-configuration) and make one.